In [11]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy as np
from music21 import instrument, note, stream, chord, interval, pitch
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

In [3]:
weightpath = "C:/Users/alecr/Projects/deepmusic/models/weights-improvement-01-0.1970-bigger.hdf5"

In [4]:
with open("C:/Users/alecr/Projects/deepmusic/pickledump/notes", 'rb') as filepath:
        notes = pickle.load(filepath)

In [5]:
type(notes)

list

In [6]:
    # Get all pitch names
pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
n_vocab = len(set(notes))

In [12]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [19]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    
    print('generating notes')
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []
    
    #a0 = network_input(start_index)

    # generate 500 notes
    for note_index in range(500):
        
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
    
        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [9]:
def create_midi(prediction_output):
    print('create_midi')
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    return stream.Stream(output_notes)

    #midi_stream.write('midi', fp='test_output.mid')

In [14]:
network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)

In [15]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
            512,
            input_shape=(network_input.shape[1], network_input.shape[2]),
            return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(weightpath)

    return model

In [16]:
model = create_network(normalized_input, n_vocab)

In [20]:
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)

generating notes


InternalError: Blas GEMM launch failed : a.shape=(1, 512), b.shape=(512, 512), m=1, n=512, k=512
	 [[{{node lstm_1/while/MatMul_7}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](lstm_1/while/Switch_3:1, lstm_1/while/MatMul_7/Enter)]]
	 [[{{node activation_1/Softmax/_111}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_765_activation_1/Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [14]:
output = create_midi(prediction_output)

create_midi


In [15]:
output.show('midi')

In [16]:
from IPython.display import Image
output.show('lily.png')
Image(filename=output.write('lily.png'))

AttributeError: 'WindowsPath' object has no attribute 'split'

In [17]:
prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
prediction_input = prediction_input / float(n_vocab)

NameError: name 'pattern' is not defined

In [18]:
pattern = network_input[numpy.random.randint(0, len(network_input)-1)]

In [29]:
pattern = np.asarray(pattern)

In [33]:
pattern = pattern.reshape(1,100,1)

In [35]:
prediction = model.predict(pattern)

In [22]:
np.asarray(network_input).shape

(57077,)

In [23]:
np.asarray(normalized_input).shape

(57077, 100, 1)

In [24]:
n = network_input[0]

In [25]:
np.asarray(n).shape

(100,)

In [26]:
n0 = n[0]

In [27]:
np.asarray(n0).shape

()

In [36]:
type(prediction_output)

list

In [37]:
np.asarray(prediction_output).shape

(500,)

In [38]:
prediction.shape

(1, 358)

In [39]:
p = prediction[0]

In [40]:
p.shape

(358,)

In [41]:
o = create_midi(p)

create_midi


TypeError: argument of type 'numpy.float32' is not iterable